# Genrating Training CSVs

這個程式的功能是從相關article把文字擷取出來，並將文字與rating放到csv檔中，方便之後用pytorch model訓練

#### 文字篩選步驟:
1. 去除含有Link:, JavaScript...等完全沒用的句子
2. 捨棄長度小於10的句子
3. 將stop words從句子裡移除
4. 計算每個字出現的次數，之後將文字由出現最多的開始加入set中，直到沒有出現次數多於3的文字或set裡含有2000個字為止
5. 將set裡的字全部輸入csv檔當作用來訓練的句子

#### 檔案位置
把這個檔案放在跟train.json, valid.json, test.json同層就能使用了，csv檔會輸出於同

In [9]:
#generate train,valid set
import json
import csv
from nltk.corpus import stopwords
from tqdm import tqdm
import time

#sentences contains these words are useless
trash = ['link:','*','javascript','twitter','____','[img]','','\\']

#stopwords to be cleared
stp = stopwords.words('english')
stp = stp+['','The','the']



with open('train.json', 'r') as f:
    with open('top2000train.csv', 'w', newline='',encoding="utf-8") as file:
        text = ""
        writer = csv.writer(file,delimiter=',')
        data = json.load(f)
        for i in tqdm(data):
            wds = []
            idx+=1
            for j in i['metadata']['premise_articles']:
                jsn = i['metadata']['premise_articles'][j]
                with open('./articles/'+jsn,'r') as f2:
                    txt = json.load(f2)
                    #get all words in the article
                    for k in txt:
                        kk = k.lower()
                        kk = kk.split(' ')#tokenize
                        if set(kk) & set(trash): #if sentence contains trash words, discard it
                            continue
                        if len(kk) < 10: #if sentence is not long enough, discard it
                            continue
                        else:
                            #add words into list
                            kk = list(filter(str.isalpha,kk))
                            kk = list(set(kk).difference(set(stp)))
                            wds = wds+kk
            #bag of words like processing
            bag = dict()
            
            #count of each words
            for j in wds:
                if j in bag:
                    bag[j] += 1
                else:
                    bag[j] = 1
            #newdict, sort by counts (big to small)
            nd={k: v for k, v in sorted(bag.items(), key=lambda item: item[1], reverse=True)}#nlogn
            #set of words in article, upmost 2000 words, add words from max count to min count
            artset = []
            for j in nd:
                if len(artset) >= 2000:
                    break
                if bag[j] >= 3:#only words with count more than 3 will be used
                    artset.append(j)
                else:
                    break
            artset.append('endblock')
            artset = ' '.join(artset)
            #append into csv
            writer.writerow([artset,str(i['label']['rating'])])
            
            

100%|████████████████████████████████████████████████████████████████████████████| 16894/16894 [33:53<00:00,  8.31it/s]


In [11]:
#same process as above to generate test set
import json
import csv
from nltk.corpus import stopwords
from tqdm import tqdm
import time

idx = 0
trash = ['link:','*','javascript','twitter','____','[img]','','\\']
stp = stopwords.words('english')
stp = stp+['','the']



with open('test.json', 'r') as f:
    with open('top2000test.csv', 'w', newline='',encoding="utf-8") as file:
        text = ""
        writer = csv.writer(file,delimiter=',')
        data = json.load(f)
        for i in tqdm(data):
            wds = []
            idx+=1
            idd=i['metadata']['id']
            for j in i['metadata']['premise_articles']:
                jsn = i['metadata']['premise_articles'][j]
                with open('./articles/'+jsn,'r') as f2:
                    txt = json.load(f2)
                    for k in txt:
                        kk = k.lower()
                        kk = kk.split(' ')
                        if set(kk) & set(trash):
                            continue
                        if len(kk) < 10:
                            continue
                        else:
                            kk = list(filter(str.isalpha,kk))
                            kk = list(set(kk).difference(set(stp)))
                            wds = wds+kk
            bag = dict()
            
            for j in wds:#n
                if j in bag:
                    bag[j] += 1
                else:
                    bag[j] = 1
            nd={k: v for k, v in sorted(bag.items(), key=lambda item: item[1], reverse=True)}#nlogn
            artset = []
            for j in nd:#n
                if len(artset) >= 2000:
                    break
                if bag[j] >= 3:
                    artset.append(j)
                else:
                    break
            artset.append('endblock')
            artset = ' '.join(artset)
            writer.writerow([artset,idd])
            
            

100%|██████████████████████████████████████████████████████████████████████████████| 2360/2360 [04:08<00:00,  9.48it/s]
